In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import mplleaflet

In [2]:
results_df=pd.read_csv(r'D:/Academics/SEM 7/Final year project/Dataset/raw.csv')
results_df.head()

,cmplnt_num,addr_pct_cd,cmplnt_fr_dt,cmplnt_fr_tm,crm_atpt_cptd_cd,juris_desc,ky_cd,law_cat_cd,loc_of_occur_desc,ofns_desc,...,pd_desc,prem_typ_desc,susp_age_group,susp_race,susp_sex,housing_psa,station_name,transit_district,hadevelopt,parks_nm
0,758521794,67,2021-03-07T00:00:00.000,01:50:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,696896951,60,2021-03-26T00:00:00.000,18:40:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,972479923,75,2021-03-27T00:00:00.000,15:55:00,COMPLETED,N.Y. POLICE DEPT,113,FELONY,FRONT OF,FORGERY,...,"FORGERY,ETC.,UNCLASSIFIED-FELO",DEPARTMENT STORE,25-44,WHITE HISPANIC,M,NaN,NaN,NaN,NaN,NaN
3,109344500,77,2021-03-28T00:00:00.000,11:10:00,COMPLETED,N.Y. POLICE DEPT,109,FELONY,FRONT OF,GRAND LARCENY,...,"LARCENY,GRAND OF VEHICULAR/MOTORCYCLE ACCESSORIES",STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,673945415,23,2021-03-29T00:00:00.000,23:13:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
results_df=results_df[['cmplnt_fr_dt','cmplnt_fr_tm','law_cat_cd','ofns_desc','latitude','longitude','boro_nm']]
results_df.head()

,cmplnt_fr_dt,cmplnt_fr_tm,law_cat_cd,ofns_desc,latitude,longitude,boro_nm
0,2021-03-07T00:00:00.000,01:50:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.654223,-73.911563,NaN
1,2021-03-26T00:00:00.000,18:40:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.575443,-73.987795,NaN
2,2021-03-27T00:00:00.000,15:55:00,FELONY,FORGERY,40.678643,-73.867543,BROOKLYN
3,2021-03-28T00:00:00.000,11:10:00,FELONY,GRAND LARCENY,40.668584,-73.926980,BROOKLYN
4,2021-03-29T00:00:00.000,23:13:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.793026,-73.942156,NaN


In [4]:
def convertClassToNum(p_df):
    p_df['datetime']=pd.to_datetime(p_df['cmplnt_fr_dt'] + ' ' + p_df['cmplnt_fr_tm'],errors='coerce')
    p_df=p_df.drop(columns=['cmplnt_fr_dt','cmplnt_fr_tm'])
    p_df=p_df.sort_values(by='datetime',ascending=True) 

    p_df['year']=pd.DatetimeIndex(p_df['datetime']).year
    p_df['month'] = pd.DatetimeIndex(p_df['datetime']).month
    p_df['weekday'] = p_df['datetime'].dt.dayofweek
    p_df['Hour'] = p_df.datetime.dt.hour
    p_df['Minute'] = p_df.datetime.dt.minute
    p_df=p_df.drop(columns=['datetime'])

    list_law_cat_cd=list(p_df['law_cat_cd'].unique())
    global dict_law_cat_cd
    dict_law_cat_cd = {}
    for i in list_law_cat_cd:
        dict_law_cat_cd[i] = list_law_cat_cd.index(i)
    p_df['law_cat_cd']=p_df['law_cat_cd'].map(dict_law_cat_cd)
#     print(dict_law_cat_cd)

    list_ofns_desc=list(p_df['ofns_desc'].unique())
    global dict_ofns_desc
    dict_ofns_desc = {}
    for i in list_ofns_desc:
        dict_ofns_desc[i] = list_ofns_desc.index(i)
    p_df['ofns_desc']=p_df['ofns_desc'].map(dict_ofns_desc)
#     print(dict_ofns_desc)

    p_df=p_df[p_df['year'].notna()]
    p_df=p_df.fillna("OTHERS")
    global dict_boro_nm
    dict_boro_nm = {'OTHERS':0,'BROOKLYN':1,'QUEENS':2,'BRONX':3,'MANHATTAN':4,'STATEN ISLAND':5}
    p_df['boro_nm']=p_df['boro_nm'].map(dict_boro_nm)
#     print(dict_boro_nm)

    return p_df


In [5]:
df_base = convertClassToNum(results_df) 

print(df_base.describe())
df_base.head()

          law_cat_cd      ofns_desc       latitude      longitude  \
count  204643.000000  204643.000000  204643.000000  204643.000000   
mean        0.657594       7.148258      40.736213     -73.923832   
std         0.750225       7.929585       0.083193       0.077488   
min         0.000000       0.000000      40.499625     -74.253767   
25%         0.000000       1.000000      40.674009     -73.970767   
50%         0.000000       6.000000      40.733990     -73.926714   
75%         1.000000      10.000000      40.811556     -73.880916   
max         2.000000      59.000000      40.912723     -73.700720   

             boro_nm           year          month        weekday  \
count  204643.000000  204643.000000  204643.000000  204643.000000   
mean        2.537355    2020.924517       3.782641       2.967084   
std         1.256488       0.923219       2.091117       1.979039   
min         0.000000    1921.000000       1.000000       0.000000   
25%         1.000000    2021.0000

,law_cat_cd,ofns_desc,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute
92521,0,0,40.748923,-73.892230,2,1921.0,1.0,6.0,8.0,50.0
95915,0,1,40.652309,-73.956290,1,1921.0,1.0,3.0,22.0,0.0
44754,0,2,40.840984,-73.925974,3,1921.0,2.0,0.0,23.0,47.0
23354,1,3,40.758800,-73.890222,2,1921.0,3.0,2.0,4.0,4.0
190767,0,4,40.511804,-74.250035,5,1955.0,1.0,5.0,0.0,1.0


In [6]:
df_columns = list()
for col in df_base.columns:
    df_columns.append(col)

print(df_columns)

['law_cat_cd', 'ofns_desc', 'latitude', 'longitude', 'boro_nm', 'year', 'month', 'weekday', 'Hour', 'Minute']


In [7]:
# inp = []
x = df_base.drop(['law_cat_cd', 'ofns_desc'],axis=1).values
y = df_base.get(['law_cat_cd', 'ofns_desc']).values

In [8]:
x

array([[ 40.74892339, -73.89223005,   2.        , ...,   6.        ,
          8.        ,  50.        ],
       [ 40.65230855, -73.95629004,   1.        , ...,   3.        ,
         22.        ,   0.        ],
       [ 40.84098357, -73.92597392,   3.        , ...,   0.        ,
         23.        ,  47.        ],
       ...,
       [ 40.66844259, -73.80061554,   2.        , ...,   2.        ,
         23.        ,  15.        ],
       [ 40.69346544, -73.92714741,   1.        , ...,   2.        ,
         23.        ,  19.        ],
       [ 40.65448439, -73.96182075,   1.        , ...,   2.        ,
         23.        ,  25.        ]])

In [9]:
y

array([[ 0,  0],
       [ 0,  1],
       [ 0,  2],
       ...,
       [ 0,  2],
       [ 1, 18],
       [ 2,  7]], dtype=int64)

In [10]:
# Split the data into x_train and y_train data sets
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, \
                                                    random_state=0,) # shuffle=False)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(163714, 8)
(163714, 2)
(40929, 8)
(40929, 2)


In [11]:
# Configure the XGBoost model
param = {'booster': 'gbtree', 
         'tree_method': 'gpu_hist',
         'n_estimators':100,
         'predictor': 'gpu_predictor',
         'max_depth': 50, 
         'eta': 0.1, 
         'objective': '{multi:softmax}', 
         'eval_metric': 'mlogloss',       # merror
         'num_round': 10,
         'feature_selector ': 'cyclic'
        }

In [12]:
def getPredictionResults(param, x_train, y_train, x_test, y_test, index):
    
    y_train = y_train.transpose()[index]
    y_test = y_test.transpose()[index]
    
    global xgb_clf 
    xgb_clf = XGBClassifier(param)
    xgb_clf.fit(x_train, y_train)
    score = xgb_clf.score(x_test, y_test)
    print("---------------------------------Index = {}-----------------------------------".format(index))
    print(score)

    xgbCpred = xgb_clf.predict(x_test)
    print(xgbCpred)

    print(y_test)

    results_log = classification_report(y_test, xgbCpred)
    print(results_log)
    
    return xgbCpred


In [13]:
law_cat_cd_pred = getPredictionResults(param, x_train, y_train, x_test, y_test, 0)

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:430: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:35:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
---------------------------------Index = 0-----------------------------------
0.5134745534950769
[0 0 1 ... 0 1 0]
[0 0 0 ... 1 1 1]
              precision    recall  f1-score   support

           0       0.52      0.93      0.67     20871
           1       0.46      0.12      0.19     13085
           2       0.33      0.01      0.01      6973

    accuracy                           0.51     40929
   macro avg       0.44      0.35      0.29     40929
weighted avg       0.47      0.51      0.40     40929



In [14]:
ofns_desc_pred = getPredictionResults(param, x_train, y_train, x_test, y_test, 1)

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:430: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:35:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
---------------------------------Index = 1-----------------------------------
0.27068826504434507
[7 0 1 ... 7 7 7]
[ 2  0  8 ...  1 18 18]
              precision    recall  f1-score   support

           0       0.33      0.62      0.43      7707
           1       0.22      0.17      0.19      4330
           2       0.19      0.19      0.19      4402
           3       0.28      0.10      0.14      3345
           4       0.69      0.87      0.77       623
           5       0.21      0.07      0.10       146
           6       0.19      0.02      0.04      1354
           7       0.23      0.47      0.31      6922
           8       0.15      0.01      0.02      1658
        

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

In [15]:
x_test

array([[ 40.59154725, -74.07930096,   5.        , ...,   6.        ,
          3.        ,   5.        ],
       [ 40.77173893, -73.95018601,   4.        , ...,   6.        ,
         13.        ,  10.        ],
       [ 40.88766381, -73.91196535,   3.        , ...,   3.        ,
          0.        ,   1.        ],
       ...,
       [ 40.668743  , -73.80779174,   2.        , ...,   3.        ,
         12.        ,  40.        ],
       [ 40.83896438, -73.8637469 ,   3.        , ...,   0.        ,
         21.        ,  56.        ],
       [ 40.82766862, -73.92543589,   3.        , ...,   4.        ,
         13.        ,  10.        ]])

In [16]:
data_pred = pd.DataFrame({'latitude':x_test[:,0] , 'longitude':x_test[:,1] , 'boro_nm':x_test[:,2] , 'year':x_test[:,3] , \
                          'month':x_test[:,4] , 'weekday':x_test[:,5] , 'Hour':x_test[:,6] , 'Minute':x_test[:,7] , \
                          'law_cat_cd_pred':law_cat_cd_pred , 'ofns_desc_pred': ofns_desc_pred})


In [17]:
print(data_pred.describe())
data_pred.head(20)

           latitude     longitude       boro_nm          year         month  \
count  40929.000000  40929.000000  40929.000000  40929.000000  40929.000000   
mean      40.736289    -73.924071      2.542012   2020.924968      3.780742   
std        0.083439      0.077851      1.256628      0.961085      2.078516   
min       40.501101    -74.250568      0.000000   1921.000000      1.000000   
25%       40.674292    -73.971869      1.000000   2021.000000      2.000000   
50%       40.734287    -73.926984      2.000000   2021.000000      4.000000   
75%       40.812116    -73.880942      4.000000   2021.000000      5.000000   
max       40.911307    -73.700720      5.000000   2021.000000     12.000000   

            weekday          Hour        Minute  law_cat_cd_pred  \
count  40929.000000  40929.000000  40929.000000     40929.000000   
mean       2.973979     13.284468     20.189719         0.090400   
std        1.976030      6.414171     18.641425         0.299756   
min        0.000

,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute,law_cat_cd_pred,ofns_desc_pred
0,40.591547,-74.079301,5.0,2021.0,3.0,6.0,3.0,5.0,0,7
1,40.771739,-73.950186,4.0,2021.0,3.0,6.0,13.0,10.0,0,0
2,40.887664,-73.911965,3.0,2021.0,3.0,3.0,0.0,1.0,1,1
3,40.610745,-73.746482,2.0,2021.0,1.0,6.0,21.0,0.0,0,0
4,40.864735,-73.905231,3.0,2021.0,1.0,5.0,18.0,25.0,0,7
5,40.627239,-73.918080,1.0,2021.0,4.0,3.0,17.0,21.0,0,7
6,40.795582,-73.932298,4.0,2021.0,5.0,5.0,10.0,7.0,0,0
7,40.759649,-73.958165,4.0,2021.0,4.0,4.0,13.0,50.0,0,0
8,40.789322,-73.813995,2.0,2021.0,3.0,5.0,0.0,25.0,0,7
9,40.828273,-73.911003,3.0,2021.0,5.0,0.0,19.0,0.0,0,0


In [18]:
data_pred["boro_nm"].unique()

array([5., 4., 3., 2., 1., 0.])

In [19]:
def convertNumToClass(p_df):
    dict_law_cat_cd1 = dict(zip(dict_law_cat_cd.values(), dict_law_cat_cd.keys()))
    p_df['law_cat_cd_pred']=p_df['law_cat_cd_pred'].astype(int)
    p_df['law_cat_cd_pred']=p_df['law_cat_cd_pred'].map(dict_law_cat_cd1)
    print(dict_law_cat_cd1)

    dict_ofns_desc1 = dict(zip(dict_ofns_desc.values(), dict_ofns_desc.keys()))
    p_df['ofns_desc_pred']=p_df['ofns_desc_pred'].astype(int)
    p_df['ofns_desc_pred']=p_df['ofns_desc_pred'].map(dict_ofns_desc1)
    print(dict_ofns_desc1)

    dict_boro_nm1 = dict(zip(dict_boro_nm.values(), dict_boro_nm.keys()))
    p_df['boro_nm']=p_df['boro_nm'].astype(int)
    p_df['boro_nm']=p_df['boro_nm'].map(dict_boro_nm1)
    print(dict_boro_nm1)

    return p_df

In [20]:
data_pred = convertNumToClass(data_pred)
print(data_pred.head())

{0: 'MISDEMEANOR', 1: 'FELONY', 2: 'VIOLATION'}
{0: 'PETIT LARCENY', 1: 'CRIMINAL MISCHIEF & RELATED OF', 2: 'ASSAULT 3 & RELATED OFFENSES', 3: 'GRAND LARCENY', 4: 'SEX CRIMES', 5: 'RAPE', 6: 'MISCELLANEOUS PENAL LAW', 7: 'HARRASSMENT 2', 8: 'OFF. AGNST PUB ORD SENSBLTY &', 9: 'MURDER & NON-NEGL. MANSLAUGHTER', 10: 'THEFT-FRAUD', 11: 'FORGERY', 12: 'BURGLARY', 13: 'VEHICLE AND TRAFFIC LAWS', 14: 'FRAUDS', 15: 'UNAUTHORIZED USE OF A VEHICLE', 16: 'FELONY ASSAULT', 17: 'KIDNAPPING & RELATED OFFENSES', 18: 'ROBBERY', 19: 'DANGEROUS DRUGS', 20: 'NYS LAWS-UNCLASSIFIED FELONY', 21: 'OTHER OFFENSES RELATED TO THEF', 22: 'PROSTITUTION & RELATED OFFENSES', 23: 'DANGEROUS WEAPONS', 24: 'OFFENSES AGAINST THE PERSON', 25: 'POSSESSION OF STOLEN PROPERTY', 26: 'GRAND LARCENY OF MOTOR VEHICLE', 27: 'OFFENSES AGAINST PUBLIC ADMINI', 28: 'CRIMINAL TRESPASS', 29: 'INTOXICATED & IMPAIRED DRIVING', 30: 'ADMINISTRATIVE CODE', 31: "BURGLAR'S TOOLS", 32: 'ARSON', 33: 'OFFENSES INVOLVING FRAUD', 34: 'NEW YORK

In [21]:
data_pred.head(20)

,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute,law_cat_cd_pred,ofns_desc_pred
0,40.591547,-74.079301,STATEN ISLAND,2021.0,3.0,6.0,3.0,5.0,MISDEMEANOR,HARRASSMENT 2
1,40.771739,-73.950186,MANHATTAN,2021.0,3.0,6.0,13.0,10.0,MISDEMEANOR,PETIT LARCENY
2,40.887664,-73.911965,BRONX,2021.0,3.0,3.0,0.0,1.0,FELONY,CRIMINAL MISCHIEF & RELATED OF
3,40.610745,-73.746482,QUEENS,2021.0,1.0,6.0,21.0,0.0,MISDEMEANOR,PETIT LARCENY
4,40.864735,-73.905231,BRONX,2021.0,1.0,5.0,18.0,25.0,MISDEMEANOR,HARRASSMENT 2
5,40.627239,-73.918080,BROOKLYN,2021.0,4.0,3.0,17.0,21.0,MISDEMEANOR,HARRASSMENT 2
6,40.795582,-73.932298,MANHATTAN,2021.0,5.0,5.0,10.0,7.0,MISDEMEANOR,PETIT LARCENY
7,40.759649,-73.958165,MANHATTAN,2021.0,4.0,4.0,13.0,50.0,MISDEMEANOR,PETIT LARCENY
8,40.789322,-73.813995,QUEENS,2021.0,3.0,5.0,0.0,25.0,MISDEMEANOR,HARRASSMENT 2
9,40.828273,-73.911003,BRONX,2021.0,5.0,0.0,19.0,0.0,MISDEMEANOR,PETIT LARCENY


In [22]:

def filter_data(data_pred, user_input = None):
    if user_input == None:
        return data_pred
    else:
        filtered_data = data_pred[data_pred['boro_nm'].str.contains(user_input)]
        return filtered_data
        print(user_input)

In [23]:
user_input = input("Enter boron name: ")
print(user_input)
m = filter_data(data_pred, user_input)
m

Enter boron name: OTHERS
OTHERS


,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute,law_cat_cd_pred,ofns_desc_pred
444,40.737082,-73.982962,OTHERS,2021.0,4.0,2.0,12.0,45.0,MISDEMEANOR,PETIT LARCENY
497,40.749654,-73.997172,OTHERS,2021.0,5.0,5.0,14.0,0.0,MISDEMEANOR,PETIT LARCENY
2186,40.722961,-74.008054,OTHERS,2020.0,12.0,5.0,12.0,0.0,MISDEMEANOR,PETIT LARCENY
2571,40.637856,-73.918438,OTHERS,2021.0,1.0,6.0,0.0,25.0,FELONY,MURDER & NON-NEGL. MANSLAUGHTER
2906,40.761865,-73.985079,OTHERS,2021.0,3.0,2.0,0.0,59.0,FELONY,PETIT LARCENY
...,...,...,...,...,...,...,...,...,...,...
38154,40.705156,-73.902122,OTHERS,2021.0,4.0,4.0,17.0,30.0,FELONY,MURDER & NON-NEGL. MANSLAUGHTER
38653,40.837934,-73.900571,OTHERS,2021.0,4.0,2.0,0.0,24.0,FELONY,MURDER & NON-NEGL. MANSLAUGHTER
38944,40.816366,-73.867626,OTHERS,2021.0,2.0,5.0,22.0,0.0,FELONY,MURDER & NON-NEGL. MANSLAUGHTER
39184,40.795582,-73.932298,OTHERS,2021.0,6.0,5.0,11.0,48.0,MISDEMEANOR,PETIT LARCENY


In [24]:
def transposeList(l1):
    return list(map(list, zip(*l1)))

In [25]:
testLat = []
testLong = []
testBoro = []
# testBoro = {}
testData = []
for key in dict_boro_nm:
    temp = filter_data(data_pred, key)
    testLat = temp['latitude'].unique()
    testLong = temp['longitude'].unique()
    testBoro = [key] * len(testLat)
#     testBoro[key] = {1: testLat,  \
#                      2: testLong}
#     temp = np.array([testLat, testLong, testBoro])
    testData += transposeList([testLat, testLong, testBoro])

In [26]:
testData

[[40.737082284000046, -73.98296229999994, 'OTHERS'],
 [40.74965446800008, -73.99717188199998, 'OTHERS'],
 [40.72296149300007, -74.00805382399994, 'OTHERS'],
 [40.63785585600005, -73.91843818899997, 'OTHERS'],
 [40.76186491100003, -73.98507852599994, 'OTHERS'],
 [40.77504198300005, -73.98206801399994, 'OTHERS'],
 [40.69579171600003, -73.939095905, 'OTHERS'],
 [40.77021126900007, -73.98224260599994, 'OTHERS'],
 [40.84416759000004, -73.91430351799994, 'OTHERS'],
 [40.81687154700006, -73.93562159799995, 'OTHERS'],
 [40.71608268700004, -73.98713051799997, 'OTHERS'],
 [40.714617135000026, -74.01144362299993, 'OTHERS'],
 [40.85826586400003, -73.89605110799994, 'OTHERS'],
 [40.714430898000046, -74.00610127799997, 'OTHERS'],
 [40.693870514000025, -73.92970371199993, 'OTHERS'],
 [40.61879293100003, -74.08204929199997, 'OTHERS'],
 [40.73634083400003, -73.91816608599999, 'OTHERS'],
 [40.86247103300008, -73.92901575199994, 'OTHERS'],
 [40.864140648000046, -73.89131303799998, 'OTHERS'],
 [40.7855183

In [27]:
len_testData = len(testData)

In [28]:
import random
from datetime import datetime

random.shuffle(testData)
testData

[[40.84926527300007, -73.90815944999997, 'BRONX'],
 [40.661076321000046, -73.95882176199996, 'BROOKLYN'],
 [40.71994386000005, -73.73332505999997, 'QUEENS'],
 [40.80407395700007, -73.93266483499998, 'MANHATTAN'],
 [40.74516029000005, -73.87330745299995, 'QUEENS'],
 [40.81428013400005, -73.81911977799997, 'BRONX'],
 [40.815669883000055, -73.93656568099993, 'MANHATTAN'],
 [40.71138986400008, -74.000106051, 'MANHATTAN'],
 [40.72558576200005, -74.000499329, 'MANHATTAN'],
 [40.69959099800008, -73.95177985899994, 'BROOKLYN'],
 [40.74412069700002, -73.70986092299995, 'QUEENS'],
 [40.76448121900006, -73.74707757199997, 'QUEENS'],
 [40.55749451600008, -74.18145105899998, 'STATEN ISLAND'],
 [40.636810336000046, -73.96068148999996, 'BROOKLYN'],
 [40.67417982200004, -73.97882534199994, 'BROOKLYN'],
 [40.67824325500004, -73.89550970099998, 'BROOKLYN'],
 [40.71394514400004, -73.73037309999995, 'QUEENS'],
 [40.83202922000004, -73.82739729899998, 'BRONX'],
 [40.58296363300008, -73.81855102299994, 'QUE

In [29]:
testTime = pd.DataFrame({'timestamp': pd.date_range(start="2021-08-01", end="2022-02-02", periods=len_testData)})
testTime['timestamp'] = testTime['timestamp'].astype(str)
testTime[['cmplnt_fr_dt','cmplnt_fr_tm']] = testTime['timestamp'].str.split(" ", 1, expand=True)
del testTime['timestamp']
testTime

,cmplnt_fr_dt,cmplnt_fr_tm
0,2021-08-01,00:00:00.000000000
1,2021-08-01,00:12:12.774033833
2,2021-08-01,00:24:25.548067666
3,2021-08-01,00:36:38.322101499
4,2021-08-01,00:48:51.096135332
...,...,...
21809,2022-02-01,23:11:08.903864670
21810,2022-02-01,23:23:21.677898502
21811,2022-02-01,23:35:34.451932336
21812,2022-02-01,23:47:47.225966168


In [30]:
testLocs = pd.DataFrame(testData, columns=['latitude', 'longitude', 'boro_nm'])
testLocs

,latitude,longitude,boro_nm
0,40.849265,-73.908159,BRONX
1,40.661076,-73.958822,BROOKLYN
2,40.719944,-73.733325,QUEENS
3,40.804074,-73.932665,MANHATTAN
4,40.745160,-73.873307,QUEENS
...,...,...,...
21809,40.755978,-73.988863,MANHATTAN
21810,40.746447,-73.830181,QUEENS
21811,40.807112,-73.912108,BRONX
21812,40.704190,-73.756067,QUEENS


In [31]:
testDataset = testTime.merge(testLocs, left_index=True, right_index=True)

In [32]:
testDataset

,cmplnt_fr_dt,cmplnt_fr_tm,latitude,longitude,boro_nm
0,2021-08-01,00:00:00.000000000,40.849265,-73.908159,BRONX
1,2021-08-01,00:12:12.774033833,40.661076,-73.958822,BROOKLYN
2,2021-08-01,00:24:25.548067666,40.719944,-73.733325,QUEENS
3,2021-08-01,00:36:38.322101499,40.804074,-73.932665,MANHATTAN
4,2021-08-01,00:48:51.096135332,40.745160,-73.873307,QUEENS
...,...,...,...,...,...
21809,2022-02-01,23:11:08.903864670,40.755978,-73.988863,MANHATTAN
21810,2022-02-01,23:23:21.677898502,40.746447,-73.830181,QUEENS
21811,2022-02-01,23:35:34.451932336,40.807112,-73.912108,BRONX
21812,2022-02-01,23:47:47.225966168,40.704190,-73.756067,QUEENS


In [33]:
# testDataset.to_csv('testDataset.csv')

In [34]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100,min_samples_split=100)
rfc.fit(x_train, y_train)

RandomForestClassifier(min_samples_split=100)

In [43]:
rfc_pred = rfc.predict(x_test)
print(classification_report(y_test[:,0],rfc_pred[:,0]))

              precision    recall  f1-score   support

           0       0.52      0.95      0.67     20871
           1       0.48      0.09      0.16     13085
           2       0.30      0.00      0.00      6973

    accuracy                           0.52     40929
   macro avg       0.43      0.35      0.28     40929
weighted avg       0.47      0.52      0.39     40929



In [44]:
print(classification_report(y_test[:,1],rfc_pred[:,1]))

              precision    recall  f1-score   support

           0       0.29      0.65      0.40      7707
           1       0.23      0.15      0.18      4330
           2       0.19      0.16      0.17      4402
           3       0.29      0.05      0.08      3345
           4       0.49      0.17      0.25       623
           5       0.00      0.00      0.00       146
           6       0.00      0.00      0.00      1354
           7       0.21      0.49      0.30      6922
           8       0.00      0.00      0.00      1658
           9       0.89      0.82      0.86        40
          10       0.29      0.06      0.09       436
          11       0.31      0.02      0.04       447
          12       0.26      0.02      0.03      1079
          13       0.00      0.00      0.00       742
          14       0.45      0.04      0.07       358
          15       0.00      0.00      0.00       124
          16       0.27      0.01      0.01      2019
          17       0.00    

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

In [48]:
# Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(x_train, y_train)

DecisionTreeClassifier()

In [49]:
dtree_pred = dtree.predict(x_test)
dtree_pred

array([[ 2,  7],
       [ 0,  0],
       [ 1, 10],
       ...,
       [ 0, 13],
       [ 0, 29],
       [ 0, 35]], dtype=int64)

In [50]:
print(classification_report(y_test[:,0],dtree_pred[:,0]))

              precision    recall  f1-score   support

           0       0.54      0.54      0.54     20871
           1       0.36      0.36      0.36     13085
           2       0.23      0.24      0.24      6973

    accuracy                           0.43     40929
   macro avg       0.38      0.38      0.38     40929
weighted avg       0.43      0.43      0.43     40929



In [51]:
print(classification_report(y_test[:,1],dtree_pred[:,1]))

              precision    recall  f1-score   support

           0       0.37      0.35      0.36      7707
           1       0.19      0.20      0.19      4330
           2       0.21      0.21      0.21      4402
           3       0.13      0.12      0.13      3345
           4       0.32      0.36      0.34       623
           5       0.07      0.06      0.07       146
           6       0.06      0.07      0.07      1354
           7       0.23      0.24      0.24      6922
           8       0.07      0.07      0.07      1658
           9       0.81      0.72      0.76        40
          10       0.08      0.09      0.08       436
          11       0.10      0.10      0.10       447
          12       0.06      0.06      0.06      1079
          13       0.04      0.04      0.04       742
          14       0.11      0.11      0.11       358
          15       0.02      0.02      0.02       124
          16       0.14      0.14      0.14      2019
          17       0.00    

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average